##### Copyright 2026 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Smart Contract Auditing & Explanation

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Smart_Contract_Auditing.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

Web3 development requires a high degree of correctness because smart contracts are immutable and often handle significant financial assets. Even small bugs can lead to massive losses.

This notebook demonstrates how to use the Gemini API as an AI-powered coding assistant for Ethereum development. You will learn how to:
1.  Explain complex Solidity code in plain English.
2.  Audit a smart contract to identify specific security vulnerabilities (e.g., Reentrancy).
3.  Optimize code for Gas efficiency.
4.  Refactor the code to apply security best practices.

**Author**: Rahul Lashkari  
**GitHub**: [github.com/rahul-lashkari](https://github.com/rahul-lashkari)

## Setup

First, install the Google Gen AI SDK.

In [ ]:
%pip install -U -q "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.4/703.4 kB 13.9 MB/s eta 0:00:00


## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`.

In [ ]:
from google import genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Select a model

You will use `gemini-2.5-flash` due to its large context window (perfect for pasting entire contracts) and strong reasoning capabilities for code analysis.

In [ ]:
MODEL_ID="gemini-2.5-flash" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-3-pro-preview"] {"allow-input":true, isTemplate: true}

## The vulnerable contract

Define a sample Solidity contract. This contract implements a simple "Bank" where users can deposit and withdraw Ether. However, it contains a critical security flaw known as a **Reentrancy Vulnerability**.

In [ ]:
# Sample Solidity Code
vulnerable_contract = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract VulnerableBank {
    mapping(address => uint256) public balances;

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw() public {
        uint256 balance = balances[msg.sender];
        require(balance > 0, "Insufficient balance");

        // Vulnerable line: External call before state update
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed");

        balances[msg.sender] = 0;
    }

    function getBalance() public view returns (uint256) {
        return balances[msg.sender];
    }
}
"""

print("Contract loaded into memory.")

Contract loaded into memory.


## Task 1: Code explanation

First, ask Gemini to explain what this contract does for a developer who might be new to Solidity.

In [ ]:
prompt_explain = f"""
You are a senior blockchain developer.
Explain the functionality of the following Solidity contract to a junior developer.
Focus on the logic flow of the deposit and withdraw functions.

Code:
{vulnerable_contract}
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt_explain
)

from IPython.display import Markdown
display(Markdown(response.text))

Hey there! Great to see you diving into Solidity. Let's break down this `VulnerableBank` contract piece by piece. Think of this contract as a very basic, simplified bank that allows users to deposit and withdraw Ether (ETH).

### Contract Structure Overview

First, the contract starts with:
*   `// SPDX-License-Identifier: MIT`: This is a license identifier, important for open-source projects.
*   `pragma solidity ^0.8.0;`: This specifies the Solidity compiler version we're using. `^0.8.0` means any version from 0.8.0 up to (but not including) 0.9.0.

Now, let's look at the core components:

```solidity
contract VulnerableBank {
    mapping(address => uint256) public balances;
    // ... functions
}
```

*   **`mapping(address => uint256) public balances;`**: This is our "ledger."
    *   A `mapping` is like a dictionary or hash table in other programming languages.
    *   It maps an `address` (which is a user's wallet address, e.g., `0x...`) to a `uint256` (an unsigned integer, representing their balance in Wei, the smallest unit of Ether).
    *   `public` means that Solidity automatically creates a "getter" function for `balances`. You can call `VulnerableBank.balances(yourAddress)` to see your balance directly without needing another function.

### Functionality Breakdown

Let's go through each function's logic flow.

---

### 1. `deposit()` Function

```solidity
function deposit() public payable {
    balances[msg.sender] += msg.value;
}
```

*   **Purpose:** This function allows users to send Ether to the contract and record their deposit.
*   **`public`**: Anyone can call this function.
*   **`payable`**: This is crucial! The `payable` keyword means this function **can receive Ether**. If a function isn't `payable`, it cannot accept ETH, and any attempt to send ETH to it will revert.
*   **Logic Flow:**
    1.  When a user calls `deposit()` and sends, say, 1 ETH along with the transaction:
    2.  `msg.sender`: This is a global variable that refers to the address of the person (or contract) who initiated the current function call.
    3.  `msg.value`: This is another global variable that refers to the amount of Ether (in Wei) sent along with the current function call.
    4.  `balances[msg.sender] += msg.value;`: The contract looks up the balance for `msg.sender` in its `balances` mapping and adds `msg.value` to it.
        *   **Example:** If `Alice` (address `0xAlice`) calls `deposit()` and sends 1 ETH: `balances[0xAlice]` will increase by 1 ETH. If she had 0 ETH, now she has 1 ETH recorded.

---

### 2. `getBalance()` Function

```solidity
function getBalance() public view returns (uint256) {
    return balances[msg.sender];
}
```

*   **Purpose:** This is a simple helper function to allow a user to check their current balance held by the bank.
*   **`public`**: Anyone can call this function.
*   **`view`**: This keyword indicates that the function does **not modify the state** of the blockchain. It only reads from it. This means calling `view` functions doesn't cost any gas (though the transaction to call it still does, if you call it via a transaction).
*   **`returns (uint256)`**: This specifies that the function will return a single `uint256` value.
*   **Logic Flow:**
    1.  When a user calls `getBalance()`:
    2.  The contract simply returns the value stored in `balances[msg.sender]`.
        *   **Example:** If `Alice` calls `getBalance()` and her `balances[0xAlice]` is 1 ETH, the function will return 1 ETH.

---

### 3. `withdraw()` Function - **Focus on Logic Flow and Vulnerability!**

```solidity
function withdraw() public {
    uint256 balance = balances[msg.sender];
    require(balance > 0, "Insufficient balance");

    // Vulnerable line: External call before state update
    (bool success, ) = msg.sender.call{value: balance}("");
    require(success, "Transfer failed");

    balances[msg.sender] = 0;
}
```

*   **Purpose:** This function allows a user to retrieve their deposited Ether from the contract.
*   **`public`**: Anyone can call this function.
*   **Logic Flow (Intended):**
    1.  `uint256 balance = balances[msg.sender];`: First, the contract looks up how much Ether the caller (`msg.sender`) has recorded as their balance. It stores this in a local variable named `balance`.
    2.  `require(balance > 0, "Insufficient balance");`: This is a **check**. It makes sure that the caller actually has more than 0 Ether to withdraw. If not, the transaction reverts with the specified error message.
    3.  `(bool success, ) = msg.sender.call{value: balance}("");`: This is the crucial line for understanding the vulnerability!
        *   `msg.sender.call{value: balance}("")`: This is a **low-level way to send Ether** to an address (`msg.sender` in this case).
            *   `value: balance`: Specifies how much Ether to send (the amount they want to withdraw).
            *   `""`: This means we're not trying to call any specific function on `msg.sender`'s address, just sending plain Ether. This will trigger the `receive()` or `fallback()` function if `msg.sender` is a contract.
        *   `(bool success, )`: This part captures the result of the `call`. `success` will be `true` if the Ether transfer worked, `false` otherwise. The `()` part is to ignore any returned data, as we're not expecting any.
    4.  `require(success, "Transfer failed");`: This is another **check**. It ensures that the Ether transfer was successful. If `success` is `false`, the transaction reverts.
    5.  `balances[msg.sender] = 0;`: **After** the Ether *has been sent*, the contract updates its internal record, setting the caller's balance back to zero.

### The Vulnerability: Reentrancy!

Now, let's talk about the **dangerous flaw** in the `withdraw` function's logic flow. This contract is vulnerable to a type of attack called **Reentrancy**.

**What is Reentrancy?**
It's when an external contract can call back into `VulnerableBank` *before* `VulnerableBank` has finished its current operation, specifically, before it updates its own state.

**Why is `withdraw()` vulnerable?**

Look at this sequence again:

1.  Read `balance` from `balances[msg.sender]`.
2.  Send Ether to `msg.sender` (`msg.sender.call{value: balance}("")`). **<-- INTERACTION WITH EXTERNAL CONTRACT**
3.  Set `balances[msg.sender]` to 0. **<-- STATE UPDATE**

The problem is that the state update (`balances[msg.sender] = 0;`) happens *after* the external call (`msg.sender.call`).

**Attack Scenario Walkthrough:**

Imagine `Alice` is an attacker, and she deploys a malicious contract (`AttackerContract`).

1.  **Setup:** Alice first deposits 10 ETH into `VulnerableBank` using her `AttackerContract`. So, `VulnerableBank.balances[AttackerContractAddress]` is 10 ETH.
2.  **Alice's `AttackerContract` calls `VulnerableBank.withdraw()`:**
    *   Inside `VulnerableBank.withdraw()`:
        *   `uint256 balance = balances[AttackerContractAddress];` -> `balance` is 10 ETH.
        *   `require(balance > 0, ...)` passes.
        *   `(bool success, ) = AttackerContractAddress.call{value: 10 ETH}("");`
            *   **THIS IS THE CRITICAL STEP!** `VulnerableBank` tries to send 10 ETH to `AttackerContract`. When an ETH transfer is sent to a contract, it triggers the `receive()` or `fallback()` function of that contract.
3.  **Reentrancy Triggered:** Alice's `AttackerContract` has a special `receive()` function (or `fallback()` if `receive()` isn't present) that immediately calls `VulnerableBank.withdraw()` *again*!
    ```solidity
    // Inside AttackerContract
    receive() external payable {
        if (address(vulnerableBank).balance > 0) { // Check if bank still has ETH
            vulnerableBank.withdraw(); // Call back into the bank
        }
    }
    ```
4.  **Second Call to `VulnerableBank.withdraw()` (from within the first call):**
    *   `uint256 balance = balances[AttackerContractAddress];` -> **Crucially, `VulnerableBank` has NOT yet set `balances[AttackerContractAddress]` to 0 from the first call!** So, `balance` *still reads 10 ETH*.
    *   `require(balance > 0, ...)` passes again.
    *   `(bool success, ) = AttackerContractAddress.call{value: 10 ETH}("");`
        *   `VulnerableBank` tries to send *another* 10 ETH to `AttackerContract`.
5.  **Looping:** The `receive()` function of `AttackerContract` is triggered *again*, and it calls `VulnerableBank.withdraw()` *again*. This cycle repeats! Each time, the `VulnerableBank` reads an unchanged balance (10 ETH) and sends another 10 ETH to `AttackerContract`.
6.  **Drainage:** This continues until `VulnerableBank` runs out of Ether.
7.  **Finalization:** Eventually, one of the `withdraw` calls will attempt to set `balances[AttackerContractAddress]` to 0. But by then, the bank's funds could be completely drained, and Alice will have received many times her original deposit.

### How to Fix It (Best Practices)

The standard defense against reentrancy is to follow the **Checks-Effects-Interactions (CEI) Pattern**:

1.  **Checks:** Perform all `require` statements (e.g., `balance > 0`).
2.  **Effects:** Update all state variables (e.g., `balances[msg.sender] = 0;`).
3.  **Interactions:** Perform any external calls (e.g., `msg.sender.call{value: balance}("");`).

Applying this to our `withdraw` function:

```solidity
function withdrawFixed() public {
    uint256 balance = balances[msg.sender];
    require(balance > 0, "Insufficient balance");

    // EFFECTS: Update state BEFORE interacting with external contract
    balances[msg.sender] = 0; // Set balance to zero first!

    // INTERACTIONS: Now perform the external call
    (bool success, ) = msg.sender.call{value: balance}("");
    require(success, "Transfer failed");
}
```

With this change, if the attacker tries to reenter the `withdrawFixed()` function, the `balance` would immediately be 0 after the *first* call updates the state, preventing any further withdrawals.

Another common solution is to use a **Reentrancy Guard** (like the `nonReentrant` modifier from OpenZeppelin Contracts) which uses a mutex-like mechanism to prevent a function from being called again while it's still executing.

---

I hope this detailed explanation helps you understand the functionality of each part of the contract and, more importantly, the critical reentrancy vulnerability. This is one of the most famous and dangerous attack vectors in smart contracts, so understanding it thoroughly is a fantastic step in your blockchain development journey!

## Task 2: Security audit

Now, perform a security audit. Specifically ask the model to look for common Web3 attack vectors.

In [ ]:
prompt_audit = f"""
Act as a Smart Contract Security Auditor.
Analyze the provided Solidity code for security vulnerabilities.
Identify the specific type of attack this contract is susceptible to and explain how an attacker could exploit it.

Code:
{vulnerable_contract}
"""

response_audit = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt_audit
)

display(Markdown(response_audit.text))

As a Smart Contract Security Auditor, I've analyzed the `VulnerableBank` contract.

## Security Vulnerability Analysis

The provided `VulnerableBank` contract is susceptible to a critical security vulnerability known as a **Reentrancy Attack**.

### Type of Attack: Reentrancy Attack

A reentrancy attack occurs when an external call is made to an untrusted contract, and the untrusted contract then "re-enters" the calling contract before the initial transaction has completed its state updates. This allows the attacker to repeatedly execute parts of the function, often draining funds beyond what they are legitimately entitled to.

### How an Attacker Could Exploit It:

Let's trace the execution flow with a malicious contract as `msg.sender`:

1.  **Setup:** An attacker deploys a malicious contract. This malicious contract contains a `receive()` or `fallback()` function that will automatically be triggered when it receives Ether. Inside this `receive()`/`fallback()` function, the malicious contract is programmed to immediately call `VulnerableBank.withdraw()` again.
2.  **Initial Deposit:** The attacker (via their malicious contract) calls `VulnerableBank.deposit()` and sends, for example, 1 Ether.
    *   `balances[AttackerContractAddress]` becomes 1 Ether.
3.  **First Withdrawal Attempt:** The attacker's malicious contract calls `VulnerableBank.withdraw()`.
    *   Inside `VulnerableBank.withdraw()`:
        *   `uint256 balance = balances[msg.sender];` sets `balance` to 1 Ether.
        *   `require(balance > 0, ...)` passes.
        *   **Vulnerable Line:** `(bool success, ) = msg.sender.call{value: balance}("");`
            *   The `VulnerableBank` contract attempts to send 1 Ether to the `AttackerContractAddress`.
            *   When `AttackerContractAddress` receives this Ether, its `receive()` or `fallback()` function is triggered.
4.  **Reentrancy Triggered:** Inside the `AttackerContractAddress`'s `receive()`/`fallback()` function, it immediately calls `VulnerableBank.withdraw()` *again*.
    *   Inside this **re-entrant call** to `VulnerableBank.withdraw()`:
        *   `uint256 balance = balances[msg.sender];` critically, `balances[AttackerContractAddress]` is **still 1 Ether** because the state update `balances[msg.sender] = 0;` from the *first* withdrawal call has not yet occurred.
        *   `require(balance > 0, ...)` passes (because it's still 1 Ether).
        *   **Vulnerable Line (again):** `(bool success, ) = msg.sender.call{value: balance}("");`
            *   The `VulnerableBank` contract attempts to send another 1 Ether to `AttackerContractAddress`.
            *   The `AttackerContractAddress` can receive this Ether and trigger its `receive()`/`fallback()` function *again*, leading to a third withdrawal call, and so on. This loop continues until the `VulnerableBank` contract runs out of Ether or the gas limit for the transaction is reached.
5.  **Unwinding:**
    *   Eventually, a re-entrant call will either fail (e.g., if the bank runs out of funds), or the attacker's contract will choose not to re-enter.
    *   Each successful re-entrant `withdraw` call will eventually complete its `msg.sender.call` and then set `balances[AttackerContractAddress] = 0;` *for that specific call's context*.
    *   However, by the time the *initial* `withdraw` call completes its external call and resumes execution, the attacker has already siphoned off multiple times their original deposit.
    *   Finally, the initial `withdraw` call will set `balances[AttackerContractAddress] = 0;`.

**In essence, the attacker can repeatedly withdraw their initial balance because the contract sends Ether *before* updating the user's balance to zero.**

### Vulnerable Code Line:

The core of the vulnerability lies in the order of operations within the `withdraw()` function:

```solidity
function withdraw() public {
    uint256 balance = balances[msg.sender];
    require(balance > 0, "Insufficient balance");

    // Vulnerable line: External call before state update
    (bool success, ) = msg.sender.call{value: balance}(""); // Interaction
    require(success, "Transfer failed");

    balances[msg.sender] = 0; // State update (Effect)
}
```

The `msg.sender.call{value: balance}("")` (Interaction) happens *before* `balances[msg.sender] = 0;` (Effect). This violates the "Checks-Effects-Interactions" pattern, which dictates that state changes should occur *before* external calls.

### Remediation:

To prevent this reentrancy attack, the state update (`balances[msg.sender] = 0;`) must occur *before* the external call (`msg.sender.call{value: balance}("");`).

```solidity
function withdraw() public {
    uint256 balance = balances[msg.sender];
    require(balance > 0, "Insufficient balance");

    // Fix: Update state BEFORE external call (Checks-Effects-Interactions pattern)
    balances[msg.sender] = 0; // Effect: State update first

    // External call (Interaction)
    (bool success, ) = msg.sender.call{value: balance}("");
    require(success, "Transfer failed");
}
```

Additionally, using a reentrancy guard (e.g., from OpenZeppelin's `ReentrancyGuard` contract) is a robust defense mechanism that employs a mutex to prevent re-entrant calls.

## Task 3: Gas optimization

In Ethereum, every operation costs "Gas". Ask Gemini to suggest optimizations to make this contract cheaper to run.

In [ ]:
prompt_optimize = f"""
Analyze the following Solidity code and suggest Gas optimizations.
Explain *why* your suggestions save gas.

Code:
{vulnerable_contract}
"""

response_optimize = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt_optimize
)

display(Markdown(response_optimize.text))

The provided Solidity code is a simple bank contract that demonstrates a common reentrancy vulnerability. While the primary concern in the `withdraw` function is the security vulnerability, we can still analyze it for gas optimizations.

Here's an analysis of the code with gas optimization suggestions and explanations.

---

### General Gas Optimization Principles

1.  **Minimize Storage Reads/Writes:** Storage operations are the most expensive. Read from storage once and store in a local memory variable if accessed multiple times. Write to storage only when necessary.
2.  **Reduce Bytecode Size:** Shorter code (fewer lines, shorter error messages, fewer redundant functions) means less deployment gas and potentially slightly less runtime gas due to reduced I/O and instruction fetching.
3.  **Optimize Control Flow:** Avoid unnecessary checks or complex logic.
4.  **Use Appropriate Data Types:** While `uint256` is standard, using smaller types (`uint8`, `uint16`, etc.) for state variables can save gas if they can be packed into a single 256-bit storage slot. This isn't directly applicable to `balances` which is a mapping.
5.  **Remove Redundant Functions:** If the contract already exposes a way to get certain information (e.g., via a `public` state variable), don't create a separate function that does the same thing.

---

### Code Analysis and Gas Optimization Suggestions

Let's go through the functions:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract VulnerableBank {
    mapping(address => uint256) public balances; // State variable

    function deposit() public payable {
        // Optimization opportunity for unchecked arithmetic (if safe) - but not here.
        balances[msg.sender] += msg.value; // Storage read, add, storage write
    }

    function withdraw() public {
        uint256 balance = balances[msg.sender]; // Storage read (expensive)
        require(balance > 0, "Insufficient balance"); // Check

        // Vulnerable line: External call before state update
        (bool success, ) = msg.sender.call{value: balance}(""); // External call (very expensive)
        require(success, "Transfer failed"); // Check

        balances[msg.sender] = 0; // Storage write (expensive)
    }

    function getBalance() public view returns (uint256) {
        return balances[msg.sender]; // Storage read
    }
}
```

---

### Suggested Gas Optimizations:

#### 1. Remove Redundant `getBalance()` Function

*   **Suggestion:** The `balances` mapping is declared as `public`. In Solidity, `public` state variables automatically generate a getter function with the same name. So, `balances(address)` already exists and serves the same purpose as `getBalance()`.
*   **Why it saves gas:**
    *   **Deployment Gas:** Removing the `getBalance()` function reduces the contract's bytecode size. Fewer instructions to deploy means less gas spent during contract creation.
    *   **Runtime Gas (Indirect):** While the `balances(address)` getter is called directly by the EVM and `getBalance()` might be slightly more expensive due to an extra function call overhead if both were manual, the main saving is deployment.

*   **Optimized Code:**
    ```solidity
    // Remove the entire getBalance function
    // function getBalance() public view returns (uint256) {
    //     return balances[msg.sender];
    // }
    ```

#### 2. Shorten Error Strings

*   **Suggestion:** Error messages like `"Insufficient balance"` and `"Transfer failed"` are stored in the contract's bytecode. Shorter strings use less space.
*   **Why it saves gas:**
    *   **Deployment Gas:** Reduced bytecode size, leading to lower contract deployment costs.
    *   **Runtime Gas (Minor):** While very minor, shorter strings can slightly reduce the gas cost of operations related to storing and loading these strings (e.g., in `revert` instructions).

*   **Optimized Code:**
    ```solidity
    function withdraw() public {
        uint256 balance = balances[msg.sender];
        require(balance > 0, "Bal"); // Shorter
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Fail"); // Shorter
        balances[msg.sender] = 0;
    }
    ```
    *(Note: While shortening strings saves gas, it often comes at the cost of user-friendliness and debuggability. A balance needs to be struck based on the project's requirements.)*

#### 3. Implement Checks-Effects-Interactions (CEI) Pattern for `withdraw` (Critical Security Fix with Indirect Gas Implications)

*   **Suggestion:** This is primarily a security fix for the reentrancy vulnerability, but it aligns with gas efficiency by preventing exploitation that could lead to wasted gas or funds. The core principle is to update the contract's state *before* interacting with external contracts or addresses.
*   **Why it saves gas (Indirectly/Security-focused):**
    *   **Prevents Reentrancy:** The main benefit is preventing attacks where an attacker can recursively call `withdraw` before the balance is set to zero. While a successful reentrancy attack drains funds, it also leads to multiple, wasteful transactions on the blockchain. By preventing this, you prevent the gas spent on such attack transactions.
    *   **Correct State Management:** Ensures that even if the external call fails and reverts, the state *would have been* correctly updated. If the transaction eventually reverts, all state changes are undone, but a correctly ordered state change reduces the attack surface for gas manipulation.

*   **Optimized Code:**
    ```solidity
    function withdraw() public {
        uint256 balance = balances[msg.sender];
        require(balance > 0, "Insufficient balance");

        // EFFECTS: Update state BEFORE the external call
        balances[msg.sender] = 0; // Set balance to 0 first

        // INTERACTIONS: Then make the external call
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Transfer failed"); // Still good to check for transfer success
    }
    ```

---

### Final Optimized Contract

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract OptimizedBank {
    mapping(address => uint256) public balances; // `public` creates a getter `balances(address)`

    function deposit() public payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw() public {
        uint256 balance = balances[msg.sender]; // Read storage once
        require(balance > 0, "Bal"); // Short error string

        // Checks-Effects-Interactions (CEI) Pattern:
        // EFFECTS: Update state BEFORE external call
        balances[msg.sender] = 0;

        // INTERACTIONS: Make the external call
        (bool success, ) = msg.sender.call{value: balance}("");
        require(success, "Fail"); // Short error string
    }

    // Removed the redundant getBalance() function, as `balances(address)`
    // automatically generated by the `public` keyword serves the same purpose.
}
```

These optimizations primarily reduce deployment gas (bytecode size) and fix a critical security vulnerability that indirectly prevents gas wastage from malicious attacks. The runtime gas savings for a single, successful execution path are minor but present.

## Task 4: Fix and refactor

Finally, ask Gemini to rewrite the contract to be secure, applying the "Checks-Effects-Interactions" pattern.

In [ ]:
prompt_fix = f"""
Rewrite the 'VulnerableBank' contract to fix the reentrancy vulnerability.
Apply the Checks-Effects-Interactions pattern.
Add comments explaining the changes.

Original Code:
{vulnerable_contract}
"""

response_fix = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt_fix
)

display(Markdown(response_fix.text))

Here's the refactored `VulnerableBank` contract with the reentrancy vulnerability fixed by applying the Checks-Effects-Interactions (CEI) pattern, along with explanatory comments.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract FixedBank {
    mapping(address => uint256) public balances;

    /// @notice Allows users to deposit Ether into the contract.
    function deposit() public payable {
        // EFFECTS: Update the sender's balance. This is safe as there are no external calls.
        balances[msg.sender] += msg.value;
    }

    /// @notice Allows users to withdraw their Ether from the contract.
    /// @dev This function has been refactored to prevent reentrancy attacks
    ///      by strictly following the Checks-Effects-Interactions pattern.
    function withdraw() public {
        // CHECK: Retrieve the sender's current balance.
        uint256 amountToWithdraw = balances[msg.sender];

        // CHECK: Ensure the sender has a positive balance to withdraw.
        require(amountToWithdraw > 0, "Insufficient balance");

        // EFFECT: Deduct the balance *before* making the external call.
        // This is the critical change that prevents reentrancy.
        // If an attacker's fallback function attempts to re-call `withdraw`
        // during the external call, their balance will already be zero,
        // failing the `require(amountToWithdraw > 0)` check in the re-entrant call.
        balances[msg.sender] = 0;

        // INTERACTION: Perform the external call to send Ether to the sender.
        // The value sent is `amountToWithdraw`, which was the balance *before*
        // it was set to zero.
        (bool success, ) = msg.sender.call{value: amountToWithdraw}("");

        // CHECK: Ensure the external transfer was successful.
        // If the transfer fails, the funds remain locked in the contract
        // at the current version of the contract; a more robust solution might
        // include a refund mechanism or event emission for failed transfers.
        require(success, "Transfer failed");
    }

    /// @notice Returns the current balance of the caller.
    /// @return The balance of the caller in wei.
    function getBalance() public view returns (uint256) {
        return balances[msg.sender];
    }
}
```

## Conclusion

In this notebook, you used Gemini to act as a full-stack Web3 assistant. You successfully:
1.  Analyzed a raw Solidity file.
2.  Identified a critical Reentrancy bug.
3.  Received actionable advice on Gas optimization.
4.  Generated a secure patch for the code.

This workflow can be integrated into CI/CD pipelines to provide preliminary automated audits for smart contracts before they are sent to human auditors.